In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [4]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'BCRA'

data = pd.read_excel(IO, sheet_name = sheet, usecols="A, L, N, P, AO", skiprows=1, nrows=1140, engine='openpyxl') # Be patient...

data = data.rename(columns={"Fechas"            :"WEEK"})
data = data.rename(columns={"Pases Pasivos"     :"PASES"})
data = data.rename(columns={"Lebacs [ARS]"      :"LEBACS"})
data = data.rename(columns={"Leliqs"            :"LELIQS"})
data = data.rename(columns={"(LEBACS+LELIQS)/BM":"LL_BM"})

data['LL_BM']  = data['LL_BM'] *100
data['PASES']  = data['PASES'] /1000000
data['LEBACS'] = data['LEBACS']/1000000
data['LELIQS'] = data['LELIQS']/1000000

In [5]:
"|CHECK DATA|"
data

,WEEK,PASES,LEBACS,LELIQS,LL_BM
0,1998-01-07,NaN,NaN,NaN,0.000000
1,1998-01-15,NaN,NaN,0.000000,0.000000
2,1998-01-23,NaN,NaN,0.000000,0.000000
3,1998-01-31,NaN,NaN,0.000000,0.000000
4,1998-02-07,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...
1135,2021-08-31,4143.395351,11.887161,4282.463851,294.323023
1136,2021-09-07,4133.693380,12.333199,4276.457453,288.672993
1137,2021-09-15,4219.983899,12.441963,4423.274413,297.387757
1138,2021-09-23,4214.801417,12.362474,4407.469149,292.919172


In [10]:
"|BUILD PLOT| ******* OLD GRAPH ******* "

cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | PASIVOS REMUNERADOS DEL BCRA",
           y_range      = (0, 4500),
           x_axis_label = "",
           y_axis_label = "Millones de pesos",
           x_range      = (dt(2015, 1,31).timestamp()*1000, dt(2021,10,31).timestamp()*1000),
           x_axis_type  = "datetime",
           plot_height  = 400,
           plot_width   = 700)

p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p.varea(y1=0, y2='LEBACS', x='WEEK', legend_label='Lebacs y Nobacs', fill_color='blue' , alpha=0.5, muted_alpha=0.2, source=cds)
p.varea(y1=0, y2='LELIQS', x='WEEK', legend_label='Leliqs'         , fill_color='gray' , alpha=0.5, muted_alpha=0.2, source=cds)
p.varea(y1=0, y2='PASES' , x='WEEK', legend_label='Pases pasivos'  , fill_color='green', alpha=0.5, muted_alpha=0.2, source=cds) 

p1 = p.line('WEEK', 'LEBACS', color='blue' , line_alpha=0.75, width=2, source=cds)
p2 = p.line('WEEK', 'LELIQS', color='gray' , line_alpha=0.75, width=2, source=cds)
p3 = p.line('WEEK', 'PASES' , color='green', line_alpha=0.75, width=2, source=cds)

p.add_tools(HoverTool(renderers=[p1], tooltips = [("Lebacs y Nobacs"  , "@LEBACS{$0,0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p2], tooltips = [("Leliqs"           , "@LELIQS{$0,0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Pases pasivos"    , "@PASES{$0,0}")] , mode="vline"))

#AXIS 2 (RIGHT)
p.extra_y_ranges = {'y2': Range1d(start=1, end=400)}
p.add_layout(LinearAxis(y_range_name='y2', axis_label='Pasivos remunerados/BM'), 'right')

p3 = p.line('WEEK', 'LL_BM', y_range_name='y2', color='red', legend_label='Pasivos/BM', line_width=2, line_alpha=0.75, muted_alpha=0.2, source=cds)
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Pasivos/BM", "@LL_BM{0.0}")], mode="vline"))

#LEGEND
#h_axis = Span(location=0, dimension="width", line_color="black")
#p.add_layout(h_axis)
p.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0")
p.legend.location     = "top_left"
p.legend.orientation  = "vertical"

p.legend.click_policy = "mute"
output_notebook()
show(p)

Loading BokehJS ...

In [7]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | PASIVOS REMUNERADOS DEL BCRA",
           y_range      = (0, 10000),
           x_axis_label = "",
           y_axis_label = "Millones de pesos",
           x_range      = (dt(2015, 1,31).timestamp()*1000, dt(2021,10,31).timestamp()*1000),
           x_axis_type  = "datetime",
           plot_height  = 400,
           plot_width   = 700)

p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p.varea_stack(['LEBACS', 'LELIQS', 'PASES'], x='WEEK', color=('blue', 'gray', 'green'), legend_label=('Lebacs', 'Leliqs', 'Pases pasivos'), alpha=0.50, source=cds)
p.vline_stack(['LEBACS', 'LELIQS', 'PASES'], x='WEEK', color=('blue', 'gray', 'green'), source=cds)

#AXIS 2 (RIGHT)
p.extra_y_ranges = {'y2': Range1d(start=1, end=350)}
p.add_layout(LinearAxis(y_range_name='y2', axis_label='Pasivos remunerados/BM'), 'right')

p3 = p.line('WEEK', 'LL_BM', y_range_name='y2', color='red', legend_label='Pasivos/BM', line_width=2, line_alpha=0.75, muted_alpha=0.2, source=cds)
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Lebacs y Nobacs"  , "@LEBACS{$0,0}"),
                                                  ("Leliqs"           , "@LELIQS{$0.0}"),
                                                  ("Pases pasivos"    , "@PASES{$0.0}" ),
                                                  ("Pasivos/BM"       , "@LL_BM{0.0}"  )], mode="vline"))

#LEGEND
#h_axis = Span(location=0, dimension="width", line_color="black")

#p.add_layout(h_axis)
p.xaxis[0].formatter = DatetimeTickFormatter(months='%b %Y')
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0")
p.legend.location     = "top_left"
p.legend.orientation  = "vertical"
p.legend.click_policy = "mute"
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="pasivos_bcra.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="pasivos_bcra.html", title="Pasivos del BCRA")
save(p)

In [8]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "pasivos_bcra"))
with open("02.06_pasivos_bcra.json", "w") as fp:
    json.dump(j, fp)